### Uncomment and run the following cells if you work on Google Colab :) Don't forget to change your runtime type to GPU!

In [1]:
# !git clone https://github.com/kstathou/vector_engine

In [2]:
# cd vector_engine

In [3]:
# pip install -r requirements.txt

### Let's begin!

In [4]:
%load_ext autoreload

In [5]:
%autoreload 2
# Used to import data from local.
import pandas as pd

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.utils import vector_search, id2details

/home/jj/anaconda3/envs/SemanticS/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Stored and processed data in s3

In [6]:
# Read a CSV in a table
df = pd.read_csv('data/100_samples_data_normalised.csv', error_bad_lines=False)

In [7]:
df.head(3)

,paperId,url,title,abstract,year,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy/0,...,authors/36/authorId,authors/36/name,authors/37/authorId,authors/37/name,authors/38/authorId,authors/38/name,authors/39/authorId,authors/39/name,fieldsOfStudy/2,fieldsOfStudy/3
0,46200b99c40e8586c8a0f588488ab6414119fb28,https://www.semanticscholar.org/paper/46200b99...,TensorFlow: A system for large-scale machine l...,TensorFlow is a machine learning system that o...,2016,94,13969,1679,False,Computer Science,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f9c602cc436a9ea2f9e7db48c77d924e09ce3c32,https://www.semanticscholar.org/paper/f9c602cc...,Fashion-MNIST: a Novel Image Dataset for Bench...,"We present Fashion-MNIST, a new dataset compri...",2017,6,4588,1340,False,Computer Science,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9c9d7247f8c51ec5a02b0d911d1d7b9e8160495d,https://www.semanticscholar.org/paper/9c9d7247...,TensorFlow: Large-Scale Machine Learning on He...,TensorFlow is an interface for expressing mach...,2016,55,9429,1008,False,Computer Science,...,145233583.0,M. Wattenberg,35078078.0,M. Wicke,2.117164e+09,Yuan Yu,2.152198e+09,Xiaoqiang Zheng,NaN,NaN


In [8]:
print(f"T'ikray Prototype: {df.paperId.unique().shape[0]}")

T'ikray Prototype: 77


The [Sentence Transformers library](https://github.com/UKPLab/sentence-transformers) offers pretrained transformers that produce SOTA sentence embeddings. Checkout this [spreadsheet](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/) with all the available models.

In this tutorial, we will use the `distilbert-base-nli-stsb-mean-tokens` model which has the best performance on Semantic Textual Similarity tasks among the DistilBERT versions. Moreover, although it's slightly worse than BERT, it is quite faster thanks to having a smaller size.

I use the same model in [Orion's semantic search engine](https://www.orion-search.org/)!

In [9]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

cuda:0


In [10]:
# Convert abstracts to vectors
embeddings = model.encode(df.abstract.to_list(), show_progress_bar=True)

Batches: 100%|████████████████████████████████████| 3/3 [00:00<00:00,  5.46it/s]


In [11]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

Shape of the vectorised abstract: (768,)


## Vector similarity search with Faiss
[Faiss](https://github.com/facebookresearch/faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, even ones that do not fit in RAM. 
    
Faiss is built around the `Index` object which contains, and sometimes preprocesses, the searchable vectors. Faiss has a large collection of [indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes). You can even create [composite indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes-(composite)). Faiss handles collections of vectors of a fixed dimensionality d, typically a few 10s to 100s.

**Note**: Faiss uses only 32-bit floating point matrices. This means that you will have to change the data type of the input before building the index.

To learn more about Faiss, you can read their paper on [arXiv](https://arxiv.org/abs/1702.08734).

Here, we will the `IndexFlatL2` index:
- It's a simple index that performs a brute-force L2 distance search
- It scales linearly. It will work fine with our data but you might want to try [faster indexes](https://github.com/facebookresearch/faiss/wiki/Faster-search) if you work will millions of vectors.

To create an index with the `misinformation` abstract vectors, we will:
1. Change the data type of the abstract vectors to float32.
2. Build an index and pass it the dimension of the vectors it will operate on.
3. Pass the index to IndexIDMap, an object that enables us to provide a custom list of IDs for the indexed vectors.
4. Add the abstract vectors and their ID mapping to the index. In our case, we will map vectors to their paper IDs from MAG.

In [50]:
#df.astype({'paperId': 'int32'}).dtypes
#df['paperId'].astype(str).astype(int)
#df['paperId'] = df.paperId.astype(int)
#df['paperId'] = pd.to_numeric(df['paperId'])
#df["paperId"] = pd.to_numeric(df["paperId"], errors='coerce')

#s = pd.Series(df['paperId'])
#print(s)
#pd.to_numeric(s, errors='coerce')


In [45]:
df.paperId

0     46200b99c40e8586c8a0f588488ab6414119fb28
1     f9c602cc436a9ea2f9e7db48c77d924e09ce3c32
2     9c9d7247f8c51ec5a02b0d911d1d7b9e8160495d
3     f9c990b1b5724e50e5632b94fdb7484ece8a6ce7
4     0090023afc66cd2741568599057f4e82b566137c
                        ...                   
72    d7701e78e0bfc92b03a89582e80cfb751ac03f26
73    218062f45c15f39bc8f4fb2c930ddf20b5809b11
74    61306b52c2d292928f7cbb2f2ef5711d15a2566c
75    62df84d6a4d26f95e4714796c2337c9848cc13b5
76    b3de1062d8a462dfdc2938558258f8884abe9f4e
Name: paperId, Length: 77, dtype: object

In [14]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.paperId.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

ValueError: did not recognize array type

### Searching the index
The index we built will perform a k-nearest-neighbour search. We have to provide the number of neighbours to be returned. 

Let's query the index with an abstract from our dataset and retrieve the 10 most relevant documents. **The first one must be our query!**


In [ ]:
# Paper abstract
df.iloc[60, 3]

In [ ]:
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[40]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nSemantic Scholar paper IDs: {I.flatten().tolist()}')

In [ ]:
# Fetch the paper titles based on their index
id2details(df, I, 'title')

In [ ]:
# Fetch the paper abstracts based on their index
id2details(df, I, 'abstract')


## Putting all together

So far, we've built a Faiss index using the misinformation abstract vectors we encoded with a sentence-DistilBERT model. That's helpful but in a real case scenario, we would have to work with unseen data. To query the index with an unseen query and retrieve its most relevant documents, we would have to do the following:

1. Encode the query with the same sentence-DistilBERT model we used for the rest of the abstract vectors.
2. Change its data type to float32.
3. Search the index with the encoded query.

Here, we will use the introduction of an article published on [HKS Misinformation Review](https://misinforeview.hks.harvard.edu/article/can-whatsapp-benefit-from-debunked-fact-checked-stories-to-reduce-misinformation/).


In [ ]:
user_query = """
WhatsApp was alleged to have been widely used to spread misinformation and propaganda 
during the 2018 elections in Brazil and the 2019 elections in India. Due to the 
private encrypted nature of the messages on WhatsApp, it is hard to track the dissemination 
of misinformation at scale. In this work, using public WhatsApp data from Brazil and India, we 
observe that misinformation has been largely shared on WhatsApp public groups even after they 
were already fact-checked by popular fact-checking agencies. This represents a significant portion 
of misinformation spread in both Brazil and India in the groups analyzed. We posit that such 
misinformation content could be prevented if WhatsApp had a means to flag already fact-checked 
content. To this end, we propose an architecture that could be implemented by WhatsApp to counter 
such misinformation. Our proposal respects the current end-to-end encryption architecture on WhatsApp, 
thus protecting users’ privacy while providing an approach to detect the misinformation that benefits 
from fact-checking efforts.
"""

In [ ]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

In [ ]:
# Fetching the paper titles based on their index
id2details(df, I, 'original_title')

In [ ]:
# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
project_dir = Path('notebooks').resolve().parents[1]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/vector_engine-master/models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)